# Xylan degradation

In this notebook we will explore xylan production in Bascillus subtilis 168

In [1]:
from spamdfba import toolkit as tk
import cobra
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import os
import warnings
import rich
import multiprocessing as mp
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2024-04-17 15:41:26,302	INFO worker.py:1724 -- Started a local Ray instance.


First we have to load the metabolic model of Bascillus subtilis 168, which is available in the BiGG database. Run below to download the model.


In [2]:
# !wget http://bigg.ucsd.edu/static/models/iYO844.json.gz
#! gzip -d iYO844.json.gz

In [3]:
from collections import Counter

Now let's load the model.

In [4]:
bascillus_model=cobra.io.load_json_model('./iYO844.json')

In [5]:
bascillus_model

Name,iYO844
Memory address,105373af0
Number of metabolites,990
Number of reactions,1250
Number of genes,844
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


We have two genes known for xylan breakdown in Bascillus subtilis 168:

Endo-1,4-beta-xylanase A:P18429

Beta-xylosidase: P94489

In [6]:
P18429="MFKFKKNFLVGLSAALMSISLFSATASAASTDYWQNWTDGGGIVNAVNGSGGNYSVNWSNTGNFVVGKGWTTGSPFRTINYNAGVWAPNGNGYLTLYGWTRSPLIEYYVVDSWGTYRPTGTYKGTVKSDGGTYDIYTTTRYNAPSIDGDRTTFTQYWSVRQSKRPTGSNATITFSNHVNAWKSHGMNLGSNWAYQVMATEGYQSSGSSNVTVW"
P94489="MKITNPVLKGFNPDPSICRAGEDYYIAVSTFEWFPGVQIHHSKDLVNWHLVAHPLQRVSQLDMKGNPNSGGVWAPCLSYSDGKFWLIYTDVKVVDGAWKDCHNYLVTCETINGDWSEPIKLNSSGFDASLFHDTDGKKYLLNMLWDHRIDRHSFGGIVIQEYSDKEQKLIGKPKVIFEGTDRKLTEAPHLYHIGNYYYLLTAEGGTRYEHAATIARSANIEGPYEVHPDNPILTSWHDPGNPLQKCGHASIVQTHTDEWYLAHLTGRPIHPDDDSIFQQRGYCPLGRETAIQKLYWKDEWPYVVGGKEGSLEVDAPSIPETIFEATYPEVDEFEDSTLNINFQTLRIPFTNELGSLTQAPNHLRLFGHESLTSTFTQAFVARRWQSLHFEAETAVEFYPENFQQAAGLVNYYNTENWTALQVTHDEELGRILELTICDNFSFSQPLNNKIVIPREVKYVYLRVNIEKDKYYYFYSFNKEDWHKIDIALESKKLSDDYIRGGGFFTGAFVGMQCQDTSGNHIPADFRYFRYKEK"

In [7]:
def get_protein_production_reaction(protein_name:str,protein_sequence:str,atp_per_aa:float=4.2)->cobra.Reaction:
    aa_name_conversion = {
        "A": "ala__L_c",
        "R": "arg__L_c",
        "N": "asn__L_c",
        "D": "asp__L_c",
        "C": "cys__L_c",
        "Q": "gln__L_c",
        "E": "glu__L_c",
        "G": "gly_c",
        "H": "his__L_c",
        "I": "ile__L_c",
        "L": "leu__L_c",
        "K": "lys__L_c",
        "M": "met__L_c",
        "F": "phe__L_c",
        "P": "pro__L_c",
        "S": "ser__L_c",
        "T": "thr__L_c",
        "W": "trp__L_c",
        "Y": "tyr__L_c",
        "V": "val__L_c",
    }
    
    counts=dict([(aa_name_conversion[i],val) for i,val in Counter(protein_sequence).items()])
    counts["atp_c"]=len(protein_sequence)*atp_per_aa
    reaction=cobra.Reaction(protein_name+"_production")
    for i,val in counts.items():
        reaction.add_metabolites({cobra.Metabolite(i):-val})
    reaction.add_metabolites({cobra.Metabolite(protein_name):1})
    reaction.add_metabolites({cobra.Metabolite("adp_c"):counts["atp_c"]})
    reaction.add_metabolites({cobra.Metabolite("pi_c"):counts["atp_c"]})
    reaction.lower_bound=0
    reaction.upper_bound=100
    ex_reaction=cobra.Reaction(protein_name+"_export")
    ex_reaction.add_metabolites({reaction.products[0]:-1})
    return [reaction,ex_reaction]
    
    

In [8]:
get_protein_production_reaction("xylanase",P18429)

[<Reaction xylanase_production at 0x3202209d0>,
 <Reaction xylanase_export at 0x320220d00>]

In [9]:
bascillus_model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.000000,-6.677343e-02
EX_drib_e,0.000000,-1.502402e-01
EX_csn_e,0.000000,-1.298372e-01
EX_5mtr_e,0.000000,-2.244329e-01
EX_dtmp_e,0.000000,-3.375768e-01
...,...,...
GND,0.000000,5.480630e-17
PGK,-3.055820,-0.000000e+00
PGLYCP,0.000000,3.469447e-18
PGM,-2.940945,-5.053628e-18


In [10]:
bascillus_model.add_reactions(get_protein_production_reaction("xylanase",P18429))

In [11]:
bascillus_model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.000000,-6.677343e-02
EX_drib_e,0.000000,-1.502402e-01
EX_csn_e,0.000000,-1.298372e-01
EX_5mtr_e,0.000000,-2.244329e-01
EX_dtmp_e,0.000000,-3.375768e-01
...,...,...
PGLYCP,0.000000,3.469447e-18
PGM,-2.940945,-0.000000e+00
PHCYT_BS,0.009060,4.998172e-17
xylanase_production,0.000000,-3.903612e+01


In [12]:
bascillus_model.objective="xylanase_production"

In [14]:
bascillus_model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.000000e+00,-3.421110e-03
EX_drib_e,0.000000e+00,-7.697498e-03
EX_csn_e,0.000000e+00,-9.503084e-05
EX_5mtr_e,0.000000e+00,-1.149873e-02
EX_dtmp_e,0.000000e+00,-8.552775e-03
...,...,...
PGLYCP,0.000000e+00,0.000000e+00
PGM,-2.692857e+00,-0.000000e+00
PHCYT_BS,-5.759422e-18,4.338842e-17
xylanase_production,6.043961e-03,4.402278e-15
